# Workflow d'un traitement de données de média social

**bjectif** : récupérer des échanges sur doctissimo pour identifier les références utilisées par les personnes sur un sujet qui revient largement sur le devant de la scène en France, l'usage médical du cannabis, et aussi l'arrivée d'une nouvelle pratique autour du CBD

## 1. Collecte des données

On veut collecter les données. On va donc construire une collecte automatique.

### Tester sur une page

Définir l'URL

In [1]:
url = "https://forum.doctissimo.fr/sante/cannabis/liste_sujet-1.htm"

Récupérer la page

In [2]:
from urllib.request import urlopen  # standard python module
html = urlopen(url).read()

On a récupéré une page

In [3]:
html[0:1000]

b'\n\t<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\t<html xml:lang="fr" lang="fr">\n\t<head>\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n\t<meta name="viewport" content="" />\n<link rel="next" href="https://forum.doctissimo.fr/sante/cannabis/liste_sujet-2.htm" />\n<title>Forum Cannabis - Doctissimo</title><link rel="stylesheet" type="text/css" href="https://forum.doctissimo.fr/forum_style.php?is_responsive=0&amp;user=0&amp;id_forum=4&amp;color_key=FFFFFF/FFFFFF/333333/C2C3F4/42529d/FFFFFF/FFFFFF/333333/333333/333333/285291/FFFFFF/F8F8F8/F8F8F8/F8F8F8/C0C0C0/C0C0C0/FFFFFF/333333/333333/0000ff/FFEEEE/000000/FFFFFF/FF0000/FFFFFF/0/0/https%3A%40%40images.doctissimo.fr%40shared%40forum/NULL/&amp;v=1638174586" /><link type="text/css" rel="stylesheet" href="https://images.doctissimo.fr/generated/forum/compressed/pictures.css?v=1638174586" /><script type="text/javascript" src="https://

C'est du HTML, il faut donc rentrer dans le détail pour récupérer les informations. On regarde le code et on voit que le tableau des sujets est repéré par la balise 'sujetCase3'

In [4]:
import bs4 # manipuler du html

html_bs4 = bs4.BeautifulSoup(html) #envelopper la page
data = html_bs4.find_all("td",{"class":"sujetCase3"}) #chercher les balises tb avec une classe spécifique
len(data)

100

On regarde ce qu'on a :

In [5]:
data[4]

<td class="sujetCase3" scope="row"><span class="DOC_cryptlinkaHR0cHM6Ly9mb3J1bS5kb2N0aXNzaW1vLmZyL3NhbnRlL2Nhbm5hYmlzL3Rlc3QtdXJpbmFpcmUtc3VqZXRfMTYyNDM5XzEuaHRt DOC_cryptlink cCatTopic" id="url_topic_162439" rel="nofollow" title="Sujet n°162439">Test urinaire thc</span></td>

In [33]:
data[6]

<td class="sujetCase3" scope="row"><a class="cCatTopic" href="https://forum.doctissimo.fr/sante/cannabis/reinsertion-socioprofessionnelle-consommateurs-sujet_162368_1.htm" id="url_topic_162368" title="Sujet n°162368">ECHANGES SUR LA RÉINSERTION SOCIOPROFESSIONNELLE D'EX-CONSOMMATEURS DE CANNABIS</a></td>

On se rend compte qu'il n'y a un lien vers le sujet que s'il a été répondu. Cela nous suffit. Sinon il faudrait une autre méthode. On récupère les liens.

Par ailleurs on est aussi intéressé à récupérer l'intitulé des sujets

Récupérer les titres

In [7]:
data[6].text

"ECHANGES SUR LA RÉINSERTION SOCIOPROFESSIONNELLE D'EX-CONSOMMATEURS DE CANNABIS"

In [9]:
textes = [i.text for i in data]

Récupérer les liens

In [11]:
# Tableau vide
liens = []

#Boucle sur les liens
for ligne in data:
    
    # Chercher le lien
    lien = ligne.find("a")
    
    # S'il existe, le garder dans le tableau
    if lien:
        liens.append(lien["href"])
        
print(len(liens))

80


Maintenant on veut récupérer toutes ces pages associées au lien pour creuser les sujets de discussion : 

In [12]:
pages = {} # tableau vide
for lien in liens[0:10]: # on teste sur 10 pour commencer
    # print(lien)
    html = urlopen(lien).read()
    pages[lien] = html

https://forum.doctissimo.fr/sante/cannabis/crise-cannabis-sujet_162359_1.htm
https://forum.doctissimo.fr/sante/cannabis/reinsertion-socioprofessionnelle-consommateurs-sujet_162368_1.htm
https://forum.doctissimo.fr/sante/cannabis/test-positif-thc-sujet_162409_1.htm
https://forum.doctissimo.fr/sante/cannabis/guerire-addiction-cannabis-sujet_162437_1.htm
https://forum.doctissimo.fr/sante/cannabis/dure-arreter-sujet_162396_1.htm
https://forum.doctissimo.fr/sante/cannabis/grosse-addiction-habitude-sujet_162436_1.htm
https://forum.doctissimo.fr/sante/cannabis/test-urinaire-sujet_151489_1.htm
https://forum.doctissimo.fr/sante/cannabis/urine-consommation-soir-sujet_162235_1.htm
https://forum.doctissimo.fr/sante/cannabis/canabis-fois-sujet_162372_1.htm
https://forum.doctissimo.fr/sante/cannabis/bouffe-delirante-sujet_162314_1.htm
https://forum.doctissimo.fr/sante/cannabis/eliminer-cannabis-urine-sujet_148548_1.htm
https://forum.doctissimo.fr/sante/cannabis/vraiment-seulement-beuh-sujet_162335_1

On a le contenu. Maintenant on veut juste récupérer les liens qui sont mentionnés par les usagers sur chaque page. Après une petite recherche, on se rend compte qu'ils sont dans une balise "a" avec une classe "cLink". On écrit donc une fonction pour les extraire

In [15]:
def obtenir_liens(html):
    liens = bs4.BeautifulSoup(html).find_all("a",{"class":"cLink"})
    return [l["href"] for l in liens]

Testons :

In [16]:
obtenir_liens(pages[list(pages.keys())[2]])

['https://cocorikush.fr/blog/29_kleaner-spray-anti-thc.html',
 'https://cocorikush.fr/blog/29_kleaner-spray-anti-thc.html']

Appliquons à toutes les pages

In [17]:
liens_pages = {}
for p in pages:
    liens_pages[p] = obtenir_liens(pages[p])

Nous avons des données un peu structurées

In [101]:
liens_pages

{'https://forum.doctissimo.fr/sante/cannabis/crise-cannabis-sujet_162359_1.htm': ['https://www.psychoactif.org/psychowiki/index.php?title=Cannabis,_effets,_risques,_t%C3%A9moignages',
  'https://www.spiritek-asso.com/tous-les-flyers/',
  'https://www.inspq.qc.ca/cannabis/cannabis-effets-psychoactifs',
  'https://www.spiritek-asso.com/tous-les-flyers/',
  'https://www.psychoactif.org/psychowiki/index.php?title=Cannabis,_effets,_risques,_t%C3%A9moignages',
  'https://www.spiritek-asso.com/tous-les-flyers/',
  'https://www.inspq.qc.ca/cannabis/cannabis-effets-psychoactifs',
  'https://www.spiritek-asso.com/tous-les-flyers/',
  'https://www.drogues-info-service.fr/',
  'https://19216811.cam/',
  'https://1921681001.id/',
  'https://19216801.onl/',
  'https://routerlogin.uno/',
  'https://192168ll.link/'],
 'https://forum.doctissimo.fr/sante/cannabis/reinsertion-socioprofessionnelle-consommateurs-sujet_162368_1.htm': ['https://idlebreakout.io/'],
 'https://forum.doctissimo.fr/sante/cannabis

## 2. Explorer et comprendre les données

On a récupéré des données. Maintenant on veut les explorer. Cela signifie passer sur des données sous la forme d'un tableau. On va utiliser une bibliothèque permettant de manipuler des tableaux, Pandas

In [18]:
import pandas as pd

In [19]:
pd.DataFrame([[p,len(liens_pages[p])] for p in liens_pages],columns=["page","nb liens"])

,page,nb liens
0,https://forum.doctissimo.fr/sante/cannabis/cri...,14
1,https://forum.doctissimo.fr/sante/cannabis/rei...,1
2,https://forum.doctissimo.fr/sante/cannabis/tes...,2
3,https://forum.doctissimo.fr/sante/cannabis/gue...,0
4,https://forum.doctissimo.fr/sante/cannabis/dur...,7
...,...,...
75,https://forum.doctissimo.fr/sante/cannabis/env...,2
76,https://forum.doctissimo.fr/sante/cannabis/ave...,1
77,https://forum.doctissimo.fr/sante/cannabis/bes...,1
78,https://forum.doctissimo.fr/sante/cannabis/com...,0


Est-ce que les liens sont uniques ?

In [20]:
# un tableau vide
tous_les_liens = []

# ajouter chaque lien de chaque page
for p in liens_pages:
    tous_les_liens+=liens_pages[p]
    
# créer un tableau et compter
pd.Series(tous_les_liens).value_counts()

https://google.com/                                                                                                                                              9
https://www.spiritek-asso.com/tous-les-flyers/                                                                                                                   9
https://www.psychoactif.org/psychowiki/index.php?title=Cannabis,_effets,_risques,_t%C3%A9moignages                                                               7
https://www.inspq.qc.ca/cannabis/cannabis-effets-psychoactifs                                                                                                    6
https://www.drogues-info-service.fr/Les-drogues-et-vous/L-arret/Est-il-possible-d-arreter-seul-le-cannabis                                                       6
                                                                                                                                                                ..
https://www.sport-equi

Certains semblent multiples. On veut regarder si cela réunit des pages ensemble. Cela nous amène à réfléchir à une mise en forme de réseau.

## 3. Mise en forme

On veut construire un réseau pour faire de l'analyse de réseau. On utilise une bibliothèque dédiée.

- chaque lien est un noeud
- on relie la page à chacun des liens

In [21]:
import networkx as nx #bibliothèque dédiée pour les réseaux

In [29]:
# Créer un graph vide
graph = nx.Graph()

# pour chaque page
for p in liens_pages:
    
    graph.add_node(p,poids=1,cat="page")
    
    # pour chaque lien de la page
    for l in liens_pages[p]:
        
        # ajouter le noeud ou augmenter sa taille
        if graph.has_node(l):
            graph.nodes[l]["poids"]+=1
        else:
            graph.add_node(l,poids=1,cat="lien")
            
        # ajouter le lien ou augmenter sa taille
        if graph.has_edge(p,l):
            graph[p][l]["poids"]+=1
        else:
            graph.add_edge(p,l,poids=1)        
        

On a maintenant construit un réseau

In [23]:
len(graph.nodes),len(graph.edges)

(127, 109)

## 4. Aller vers la finalisation des données

**Attention il faut une bibliothèque spécifique : pip install ipysigma**

On a un réseau, on veut le visualiser. On peut sortir les données sous un format utilisable sous Gephi. Sinon il est possible de le regarder directement.

In [24]:
from ipysigma import Sigma

Créer le réseau

In [30]:
Sigma(graph,node_size="poids",node_color="cat")

Sigma(nx.Graph with 164 nodes and 109 edges)

## Questions pour le script

- comment faire pour récupérer plus de pages ?
- comment l'appliquer à un autre sujet ?
- comment faire pour savoir le nombre de sujets qui parlent de CBD ?